In [4]:

import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt # NOTE: This was tested with matplotlib v. 2.1.0
 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [5]:
#########################
#
# Data Import
#
#########################
df = pd.read_csv('../../_data/operations_imputed_CLEAN.csv', index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91862 entries, 0 to 102290
Data columns (total 64 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   op_id              91862 non-null  int64  
 1   subject_id         91862 non-null  int64  
 2   hadm_id            91862 non-null  int64  
 3   opdate             91862 non-null  int64  
 4   age                91862 non-null  int64  
 5   sex                91862 non-null  object 
 6   weight             90910 non-null  float64
 7   height             91326 non-null  float64
 8   race               91862 non-null  object 
 9   asa                89532 non-null  float64
 10  emop               91862 non-null  int64  
 11  department         91862 non-null  object 
 12  antype             91862 non-null  object 
 13  icd10_pcs          91862 non-null  object 
 14  category_desc      91862 non-null  object 
 15  desc_short         91862 non-null  object 
 16  category_id        91862 n

In [10]:
#########################
#
# Drop Target Column, NaN columns and excluded Features
#
#########################
# 
## Drop cols with many NaN

columns_w_NaN = ['art_dbp']  # Define as a list

## Drop Excluded Features
cols_to_drop = columns_w_NaN + ['op_id', 'subject_id', 'hadm_id', 'opdate', 'is_outlier', 'category_desc', 'desc_short', 'subject_id_y', 'inhosp_death_time', 'orin_time', 'orout_time', 'opstart_time', 'opend_time', 'admission_time', 'discharge_time', 'anstart_time', 'anend_time', 'cpbon_time', 'cpboff_time', 'icuin_time', 'icuout_time', 'icd10_pcs', 'chart_time_x', 'race', 'emop']  # Include 'art_dbp' directly


# Check if the columns exist in the DataFrame before dropping them
missing_columns = [col for col in cols_to_drop if col not in df.columns]
if missing_columns:
    print("Columns not found in DataFrame:", missing_columns)

df = df.drop(columns=cols_to_drop, axis=1)

# Drop Targets
y_target_reg = 'LOS'

# Confirm
x_df = df.drop([y_target_reg], axis=1)

# Check the info of the resulting DataFrame
x_df.info()

# Display the resulting DataFrame
print(x_df)

<class 'pandas.core.frame.DataFrame'>
Index: 91862 entries, 0 to 102290
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              91862 non-null  int64  
 1   sex              91862 non-null  object 
 2   weight           90910 non-null  float64
 3   height           91326 non-null  float64
 4   asa              89532 non-null  float64
 5   department       91862 non-null  object 
 6   antype           91862 non-null  object 
 7   category_id      91862 non-null  object 
 8   art_mbp          89504 non-null  float64
 9   art_sbp          89497 non-null  float64
 10  bt               89479 non-null  float64
 11  cvp              89248 non-null  float64
 12  hr               89515 non-null  float64
 13  pip              89519 non-null  float64
 14  pmean            89508 non-null  float64
 15  rr               89517 non-null  float64
 16  spo2             89517 non-null  float64
 17  vt              

In [11]:
x_df.dropna()


,age,sex,weight,height,asa,department,antype,category_id,art_mbp,art_sbp,...,lymphocyte,platelet,potassium,sodium,total_bilirubin,wbc,prolonged_LOS,icu_visit,or_duration,anesth_duration
3,50,F,66.0,157.0,2.0,OS,General,0NQ,100.826720,137.244265,...,20.414394,223.140988,3.768724,139.116926,0.851504,8.957105,1,0,45.0,70.0
4,60,F,62.0,154.0,1.0,GS,General,0GT,100.773779,138.601562,...,23.938716,217.282759,3.846584,140.033084,0.744921,8.200501,1,0,70.0,90.0
5,35,F,50.0,160.0,1.0,OS,Neuraxial,09B,98.084871,131.646617,...,13.300000,124.000000,3.900000,138.000000,0.600000,6.310000,0,0,115.0,150.0
6,20,M,62.0,179.0,1.0,OL,General,09Q,92.702290,140.826772,...,22.134112,237.222222,4.041013,139.824013,1.078906,10.044925,1,0,90.0,135.0
7,60,F,52.0,152.0,1.0,OL,General,09Q,100.773779,138.601562,...,23.938716,217.282759,3.846584,140.033084,0.744921,8.200501,0,0,30.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102286,50,F,58.0,162.0,2.0,GS,General,0DN,100.826720,137.244265,...,20.414394,223.140988,3.800000,134.000000,0.851504,8.957105,0,0,160.0,180.0
102287,70,F,53.0,162.0,2.0,GS,General,0HB,99.036609,142.337732,...,21.154115,212.690065,3.700000,142.000000,0.760224,8.535719,1,0,60.0,75.0
102288,65,F,51.0,152.0,2.0,GS,General,0HB,97.106931,138.279011,...,21.422042,212.006881,3.800000,143.000000,0.780793,8.557983,1,0,55.0,75.0
102289,85,M,74.0,171.0,4.0,GS,General,0DB,92.000000,152.000000,...,19.733333,218.800000,3.900000,137.000000,1.800000,9.932500,1,1,130.0,170.0


In [ ]:
#########################
#
# One-Hot encode categorical Features
#
#########################

# Filter columns with dtype 'object'
object_columns = x_df.select_dtypes(include='object').columns.tolist()
print(object_columns)



In [ ]:
from sklearn.preprocessing import OneHotEncoder
## use OneHot instead of get_dummies
        
def encode_cat_vars(x):
    original_cols = set(x.columns)
    
    # Extract the object columns
    object_columns = [col for col in x.columns if x[col].dtype == 'object']
    
    # Initialize the OneHotEncoder
    encoder = OneHotEncoder(drop='first', sparse=False)
    
    # Fit and transform the encoder on the object columns
    encoded_data = encoder.fit_transform(x[object_columns])
    
    # Create a DataFrame from the encoded data with appropriate column names
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(object_columns))
    
    # Drop the original object columns
    x = x.drop(object_columns, axis=1)
    
    # Concatenate the encoded DataFrame with the original DataFrame
    x = pd.concat([x, encoded_df], axis=1)
    
    # Get the newly added columns
    dummy_cols = list(set(x.columns) - original_cols)
    
    return x, dummy_cols

# Dummy variable creation is done before splitting the data, so all the different categories are covered
# Create dummy variables
df_encoded, dummy_columns = encode_cat_vars(x_df)

#confirm:
# df_encoded = df_encoded.dropna(subset=['LOS'])
df_encoded.shape
df_encoded.to_csv('encoded.csv')

With current categories- there are 164 features after 1-ht. This may be a problem. 
Need to remember that each model will be trained on the category 

In [ ]:
df = df.dropna(subset=['prolonged_LOS'])

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.decomposition import PCA
#########################
#
# Principal Component Analysis - PCA - Demographics
#
#########################
# 
target_column = y_target_cat  # Replace with the y_regression if want to change model to regression. 
object_columns = [col for col in df_encoded.columns if col != target_column]

# Initialize the HistGradientBoostingClassifier for feature selection
gbdt = HistGradientBoostingClassifier(random_state=0)
gbdt.fit(df_encoded[object_columns], df_encoded[target_column])

# Get feature importances from the trained GBDT model
feature_importances = gbdt.feature_importances_

# Sort features by importance in descending order
sorted_indices = feature_importances.argsort()[::-1]

# Select the top N most important features (you can adjust N as needed)
num_selected_features = 3  # revise thise following our skree plot (below)
selected_feature_indices = sorted_indices[:num_selected_features]

# Extract the selected features from your DataFrame
selected_features = df_encoded.iloc[:, selected_feature_indices]

# Now, you can perform PCA on the selected features
scaled_data = selected_features.values  # Convert to NumPy array
pca = PCA()
pca.fit(scaled_data)
pca_data = pca.transform(scaled_data)

In [ ]:
#########################
#
# Troubleshoot any NaN values in PCA
#
#########################

# Check for NaN values in the 'y' column
nan_indices = df_encoded[df_encoded.columns].isna()
print(nan_indices)

In [ ]:

#########################
#
# Draw a scree plot and a PCA plot
#
#########################
 
import numpy as np
import matplotlib.pyplot as plt


#The following code constructs the Scree plot
per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]
 
plt.bar(x=range(1,len(per_var)+1), height=per_var, tick_label=labels)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.title('Scree Plot')
plt.show()
 
#the following code makes a fancy looking plot using PC1 and PC2
pca_df = pd.DataFrame(pca_data, index=[*wt, *ko], columns=labels)
 
plt.scatter(pca_df.PC1, pca_df.PC2)
plt.title('My PCA Graph')
plt.xlabel('PC1 - {0}%'.format(per_var[0]))
plt.ylabel('PC2 - {0}%'.format(per_var[1]))
 
for sample in pca_df.index:
    plt.annotate(sample, (pca_df.PC1.loc[sample], pca_df.PC2.loc[sample]))
 
plt.show()
 
#########################
#
# Determine which genes had the biggest influence on PC1
#
#########################
 
## get the name of the top 10 measurements (genes) that contribute
## most to pc1.
## first, get the loading scores
loading_scores = pd.Series(pca.components_[0], index=genes)
## now sort the loading scores based on their magnitude
sorted_loading_scores = loading_scores.abs().sort_values(ascending=False)
 
# get the names of the top 10 genes
top_10_genes = sorted_loading_scores[0:10].index.values
 
## print the gene names and their scores (and +/- sign)
print(loading_scores[top_10_genes])